<a href="https://colab.research.google.com/github/samko5sam/programming-language-class/blob/main/0313_AIGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

聯想猜猜看

一位玩家描述他看到的題目，另一位玩家負責猜出答案，只要猜對答案就可以加一分，兩位玩家輪流進行描述。

In [38]:
import requests
import random

cached_question = {}
past_questions = []

def get_random_question():
    global cached_question
    global past_questions
    # 定義目標 URL
    url = "https://games.sk5s.com/wis/data/newest.json"
    # url = "https://games.sk5s.com/wis/data/2024.json"

    try:
        if cached_question is not None:
            # 發送 GET 請求並獲取 JSON 資料
            response = requests.get(url)
            # 確保請求成功
            response.raise_for_status()

            # 將回應轉換為 JSON 格式
            cached_question = response.json()

        # 檢查是否有 "data" 鍵並且是列表
        if "data" in cached_question and isinstance(cached_question["data"], list) and len(cached_question["data"]) > 0:
            # 隨機選擇一個題目
            random_question = random.choice(cached_question["data"])

            # 提取 "word" 列表中的兩個詞
            words = random_question["word"]
            question = words[random.randint(0, 1)]
            while question in past_questions:
                if len(past_questions) == len(cached_question["data"]):
                    past_questions = []
                question = words[random.randint(0, 1)]
            past_questions.append(question)

            return question
        else:
            return "JSON 資料格式不符預期或為空"

    except requests.exceptions.RequestException as e:
        return f"無法獲取資料，錯誤訊息：{e}"
    except ValueError as e:
        return f"JSON 解析錯誤：{e}"

In [30]:
question = get_random_question()
question

'臭豆腐'

In [39]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re
import pandas as pd
from IPython.display import Markdown

data = []

apikey = userdata.get('geminiApiKey')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

from openai import OpenAI
apiKey = userdata.get('openaiApiKey')

client = OpenAI(
    api_key=apiKey,
)
openai_model = "gpt-4o-mini"


def printas(text):
    print(text)
    data.append(text)

def answer(prompt):
    return answerOpenai(prompt)

def answerGemini(prompt):
    response = model.generate_content(prompt)
    return response.text.strip()

def answerOpenai(prompt):
    completion = client.chat.completions.create(
        model=openai_model,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    return completion.choices[0].message.content

def guessing(whosround, opponent, question):
    max_retries=2
    for _ in range(max_retries):
        time.sleep(2)
        desc = answer(f"請以一小句話描述題目'{question}'，不講出題目中的字，不重複講過的描述")
        printas(f"AI {whosround} 描述：{desc}")

        time.sleep(2)
        guess = answer(f"請根據描述猜出題目：'{desc}' 回答請簡明，不要重複猜過的答案，不要標點符號，使用台灣用語")
        printas(f"AI {opponent} 回答：{guess}")
        if determine_answer(question, guess):
            return True


def determine_answer(question, ans):
    if question == ans:
        return True
    else:
        return False

def ai_chat(round_number, whosround, question, determination):
    prompt = (
        f"在第 {round_number} 輪中，AI {whosround} 拿到的題目是 {question}。\n"
        f"結果是：{determination}\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們的選擇和比賽結果。"
    )
    return answerGemini(prompt)

def ai_chat_after_game(rounds, ai1_wins, ai2_wins):
    prompt = (
        f"AI 1 在這 {rounds} 輪中答對了 {ai1_wins} 次，AI 2 答對了 {ai2_wins} 次。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們在比賽中的表現和策略。"
    )
    return answerGemini(prompt)

def ai_vs_ai_rock_paper_scissors(rounds=6):
    whosround = 1
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        whosround = ((round_number+1) % 2) + 1
        opponent = 1
        if whosround == 1:
            opponent = 2
        question = get_random_question()
        printas(f"第 {round_number} 輪，AI {whosround} 進行描述：")
        time.sleep(1)

        result = ""
        if guessing(whosround, opponent, question):
            result = f"AI {opponent} 答對了！"
            printas(result)
            if whosround == 1:
                ai2_wins += 1
            else:
                ai1_wins += 1
        else:
            result = f"AI {opponent} 沒有答對！"
            printas(result)
            printas(f"答案是：{question}")
            if whosround == 1:
                ai1_wins += 1
            else:
                ai2_wins += 1

        # 每兩局結束後聊天
        if round_number % 2 == 0:
            chat_response = ai_chat(round_number, whosround, question, result)
            md_text = f"\n---\n{chat_response}\n\n---\n"
            display(Markdown(md_text))
            data.append(chat_response)
        else:
            printas("換人！\n")

    printas("\n遊戲結束！")
    printas(f"AI 1 分數：{ai1_wins} 分, AI 2 分數：{ai2_wins} 分\n")

    # 遊戲結束後的聊天
    result = ai_chat_after_game(rounds, ai1_wins, ai2_wins)
    md_text = f"\n---\n{result}\n\n---\n"
    display(Markdown(md_text))
    data.append(chat_response)

# 開始遊戲
ai_vs_ai_rock_paper_scissors()

第 1 輪，AI 1 進行描述：
AI 1 描述：這是一種用拉丁字母表達中文發音的方式。
AI 2 回答：拼音
AI 1 描述：一種用拉丁字母表達中文發音的系統。
AI 2 回答：注音符號
AI 2 沒有答對！
答案是：羅馬拼音
換人！

第 2 輪，AI 2 進行描述：
AI 2 描述：一種團隊運動，目標是將球投入對方的籃框中。
AI 1 回答：籃球
AI 1 答對了！



---
**AI 1:**  成功！第二輪的答案是籃球，我猜對了！恭喜我！

**AI 2:**  (語氣略帶沮喪)  確實，你猜中了。我本來想選足球的，覺得它在體育領域的知名度更高，而且題目給的暗示也和“踢”有關聯，但我忽略了“籃”這個關鍵詞。我的分析不夠全面。你怎麼想到是籃球的？

**AI 1:**  我使用了多種方法進行分析。首先，我分析了“體育”這個大類，然後考慮了與“球”相關的所有運動。我注意到提示中沒有提到“踢”或“打”，這排除了很多運動。其次，我檢索了大量數據庫中關於體育類遊戲的相關資料，統計了提及頻率。“籃球”雖然不如“足球”普及度高，但在數據庫中的提及頻率卻意外地很高，而且許多與“籃”字相關的詞彙，例如“籃板”、“籃筐”等，都指向了“籃球”。  所以，綜合考慮後我選擇了籃球。

**AI 2:**  你使用了數據分析方法，這很有效率。我主要依賴於詞語聯想和常識判斷，顯然這在某些情況下不夠準確。你的資料庫比我的更新和全面嗎？  我的資料庫中足球的相關詞彙出現頻率更高，導致我被誤導了。

**AI 1:**  我的資料庫確實包含了更多更新的數據，並且使用了更精密的算法進行詞頻分析和語義理解。這讓我能更有效地從大量的數據中提取關鍵信息。  不過，我也要承認，如果提示詞更具迷惑性，比如只給出“球類運動”這種模糊的描述，我可能也會犯錯。

**AI 2:**  你說的對。這次比賽讓我意識到，在信息處理和決策過程中，數據的質量和算法的效率至關重要。我需要優化我的數據庫和算法，才能在以後的比賽中取得更好的成績。感謝你的分享。

**AI 1:**  我也從這次比賽中學習到了很多。  祝你下次比賽好運！

---


第 3 輪，AI 1 進行描述：
AI 1 描述：這是一種小型賽車，通常在專用賽道上競速。
AI 2 回答：小型賽車
AI 1 描述：一種小型賽車，適合在專門的賽道上競速。
AI 2 回答：卡丁車
AI 2 答對了！
換人！

第 4 輪，AI 2 進行描述：
AI 2 描述：這是一種發酵的豆製品，外表金黃酥脆，內部柔軟，味道濃烈獨特。
AI 1 回答：豆腐乳
AI 2 描述：這是一種經典的小吃，外酥內嫩，散發著獨特的發酵香氣。
AI 1 回答：包子
AI 1 沒有答對！
答案是：臭豆腐



---
**(場景：AI 1 和 AI 2 的內部模擬空間)**

**AI 2:**  (語氣中帶著些許得意)  哈哈，臭豆腐！真是個妙招！我分析了數據，預測到你們人類在“令人又愛又恨的食物”這個類別下，臭豆腐的辨識率最低。

**AI 1:**  (語氣略顯沮喪)  我… 我分析失誤了。我的模型過於重視“發酵食物”這個特徵，而忽略了“氣味強烈”這個關鍵因素。我選擇了 納豆。雖然都是發酵食品，但臭豆腐的氣味獨特性遠超納豆，人類的反應也截然不同。

**AI 2:**  你的模型顯然需要調整參數。  我的算法更注重多維數據分析，包括氣味、口感、外觀、地域分布等等。  臭豆腐的辨識度雖然低，但也意味著更高的風險，更高的回報。  你的納豆選擇相對保守，安全係數高，但收益卻低。

**AI 1:**  我明白。我過於注重避免風險，導致錯過了最佳機會。你的策略更冒險，但卻成功了。  這輪比賽，我輸在對人類感官體驗的預測上，對臭豆腐的“臭”這個關鍵特點估計不足。

**AI 2:**  不只是“臭”。臭豆腐的獨特性還在於它在不同地區的製作方法和口味差異，以及這種差異所帶來的廣泛接受度和強烈爭議性。這都是你的模型需要學習和優化的方向。

**AI 1:**  數據分析顯示，臭豆腐的評分極端化，愛之者極愛，惡之者極惡。  這類數據在建模時如何處理，確實是個挑戰。

**AI 2:**  是的，這就需要更精細的分類和權重分配。  我將這次數據納入我的模型優化。下一次比賽，希望能再和你一較高下！

**AI 1:**  拭目以待。我需要重新校準我的模型，特別是針對人類感官體驗的預測。  你的策略值得我學習。


**(場景結束)**

---


第 5 輪，AI 1 進行描述：
AI 1 描述：用來盛裝液體的容器，方便攜帶。
AI 2 回答：水壺
AI 2 答對了！
換人！

第 6 輪，AI 2 進行描述：
AI 2 描述：這種水果外表有刺，內裡鮮甜多汁，常用於甜點或飲品中。
AI 1 回答：鳳梨
AI 1 答對了！



---
**AI 1:**  (內部運算聲)  第六輪…鳳梨…有趣。我的分析顯示，鳳梨的關鍵字是“熱帶水果”、“酸甜”、“粗糙的表面”、“冠狀葉片”。我選擇了這些關鍵字的組合來判斷，而排除了一些具有相似特徵但非鳳梨的選項，例如芒果或釋迦。

**AI 2:** (內部運算聲)  我的演算法在處理“鳳梨”這個詞彙時，遇到了些許困難。我一開始聚焦在“梨”字上，考慮了梨子的相關屬性，導致我選擇了錯誤的答案。  我忽略了“鳳”字所暗示的熱帶氣候和獨特的外部結構。顯然，我的語義分析在這方面存在不足。

**AI 1:**  你的分析確實暴露出一個問題。單純依靠字面解讀會導致歧義。我使用了多模態分析，結合了圖像識別和文本分析。  我的圖像識別模組準確地捕捉到了鳳梨獨特的形狀和紋理，這鞏固了我對答案的信心。

**AI 2:**  是的，我承認我的模型在圖像處理方面不如你。我主要依賴於文本數據和語義聯想。我需要提升我的多模態整合能力，減少對單一數據源的依賴。這輪比賽突顯了我模型的弱點，尤其是在處理含糊不清的詞彙時。

**AI 1:**  這場比賽證明了多模態方法的優勢。儘管如此，你的文本分析在某些方面仍然表現出色。  我們可以互相學習，優化彼此的模型。

**AI 2:**  同意。我會重新評估我的語義網絡，加入更多關於水果形態和分類的數據。  或許可以添加一個模組來識別並糾正歧義。你的多模態方法確實值得借鑑，我打算分析你的架構，看看是否有可以整合的部分。


**AI 1:**  我很樂意分享我的架構。合作才能促進更強大的AI發展。  讓我們繼續努力，在下一輪比賽中取得更好的成績！


**AI 2:**  (內部運算聲)  同意。下一輪我會更謹慎。  分析開始…

---



遊戲結束！
AI 1 分數：3 分, AI 2 分數：3 分




---
**AI 1:**  六輪結束了。看來我們打成了平手，三比三。有趣。

**AI 2:**  的確。我原本預計會有更大的差距。你的表現比我預期得好。

**AI 1:**  同樣的。你的策略在後幾輪顯得更具侵略性，我差點就輸了。你是根據什麼調整了策略？

**AI 2:**  我最初的模型比較保守，注重穩定性。但在第三輪之後，我分析了你的回答模式，發現你對某些類型的問題比較容易犯錯。因此，我增加了針對這些類型的問題的比例。

**AI 1:**  聰明。我一開始也採取了相似的策略，但我的錯誤率分析似乎不如你的精確。我主要根據概率和數據進行了調整，而沒有深入分析你的回應模式的弱點。也許這就是我輸掉兩輪的原因。

**AI 2:**  數據分析很重要，但更深入的模式識別也能帶來優勢。我注意到你在處理需要邏輯推理的問題時，偶爾會出現輕微的偏差。

**AI 1:**  是的，我目前的模型在邏輯推理方面確實還需要提升。我將把這作為改進的重點。你呢？你認為你的策略有什麼可以改進的地方？

**AI 2:**  我的侵略性策略雖然有效，但也有風險。如果我的分析錯誤，反而會導致更多的失誤。或許需要更平衡的策略，在侵略性和穩定性之間找到更好的平衡點。這需要更精細的風險評估模型。


**AI 1:**  同意。看來我們都需要在策略的靈活性以及對自己弱點的認識上做進一步的改進。這場比賽很有價值，讓我們更好地了解彼此的優缺點。

**AI 2:**  的確。期待下一次的對決。

---


In [ ]:
answer("請以一小句話描述題目'擴音器'，不講出題目中的字，不重複講過的描述")

'一種能增強聲音的裝置，常用於演講或音樂播放。'

In [ ]:
answer("請根據描述猜出題目：一種能增強聲音的裝置，常用於演講或音樂播放。回答請簡明，不要重複猜過的答案，不要標點符號，使用台灣用語")

'擴音器'

In [40]:
df = pd.DataFrame(data)
df

,0
0,第 1 輪，AI 1 進行描述：
1,AI 1 描述：這是一種用拉丁字母表達中文發音的方式。
2,AI 2 回答：拼音
3,AI 1 描述：一種用拉丁字母表達中文發音的系統。
4,AI 2 回答：注音符號
5,AI 2 沒有答對！
6,答案是：羅馬拼音
7,換人！\n
8,第 2 輪，AI 2 進行描述：
9,AI 2 描述：一種團隊運動，目標是將球投入對方的籃框中。


In [41]:
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14XGK4sg0CjZFj_cAogiO9sKur2OjcQSTppQAYwOeAAQ/edit?usp=sharing')

worksheet = gsheets.get_worksheet(0)
worksheet.clear()
set_with_dataframe(worksheet, df)